In [ ]:
%pip install camel-tools

from google.colab import drive
import os

drive.mount('/gdrive')
os.environ['CAMELTOOLS_DATA'] = '/gdrive/MyDrive/camel_tools'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 kB 44.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for camel-kenlm: filename=camel_kenlm-2023.3.17.2-cp310-cp310-linux_x86_64.whl size=3453097 sha256=e8b9606c896c276eda5adb65e69e5a8fb460659fcad49bc2d9ec63f807d96606
  Stored in directory: /root/.cache/pip/wheels/29/c5/32/09633c3b70fdfc470b2fb912bd9e90d8d6814df68c794dcaa6
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=44e1fb1b139dedd69ef8c826e7aa1f16d6

In [ ]:
from camel_tools.utils.normalize import normalize_unicode
from camel_tools.utils.dediac import dediac_ar
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.tokenizers.morphological import MorphologicalTokenizer
from collections import defaultdict
import math
import pandas as pd
import re
import unicodedata
from nltk import FreqDist
from nltk import ngrams

In [ ]:
df1 = pd.read_csv('//gdrive/MyDrive/AI Project - Classifying Machine-Translated And Human-Translated Arabic Text/New dataset with test/ar_en_dev_filtered.csv')
df2 = pd.read_csv('//gdrive/MyDrive/AI Project - Classifying Machine-Translated And Human-Translated Arabic Text/New dataset with test/ar_en_tune_filtered.csv')
df3 = pd.read_csv('//gdrive/MyDrive/AI Project - Classifying Machine-Translated And Human-Translated Arabic Text/New dataset with test/ar_fr_dev_filtered.csv')
df4 = pd.read_csv('//gdrive/MyDrive/AI Project - Classifying Machine-Translated And Human-Translated Arabic Text/New dataset with test/ar_fr_tune_filtered.csv')


In [ ]:
def melt_and_label(df, mt_column, ht_column):
    mt_melted = pd.melt(df, value_vars=[mt_column], var_name='column', value_name='text')
    mt_melted['labels'] = 'MT'

    ht_melted = pd.melt(df, value_vars=[ht_column], var_name='column', value_name='text')
    ht_melted['labels'] = 'HT'

    melted_df = pd.concat([mt_melted, ht_melted], ignore_index=True)

    return melted_df

df1_melted = melt_and_label(df1, 'ar_mt', 'ar_ht')
df2_melted = melt_and_label(df2, 'ar_mt', 'ar_ht')
df3_melted = melt_and_label(df3, 'ar_mt', 'ar_ht')
df4_melted = melt_and_label(df4, 'ar_mt', 'ar_ht')

concatenated_df = pd.concat([df1_melted, df2_melted, df3_melted, df4_melted], ignore_index=True)

In [ ]:
test_df1 = pd.read_csv('/gdrive/MyDrive/AI Project - Classifying Machine-Translated And Human-Translated Arabic Text/New dataset with test/ar_fr_test.csv')
test_df2 = pd.read_csv('/gdrive/MyDrive/AI Project - Classifying Machine-Translated And Human-Translated Arabic Text/New dataset with test/ar_en_test.csv')

In [ ]:
test_df1_melted = melt_and_label(test_df1, 'ar_mt', 'ar_ht')
test_df2_melted = melt_and_label(test_df2, 'ar_mt', 'ar_ht')

test_df = pd.concat([test_df1_melted, test_df2_melted], ignore_index=True)

In [ ]:
mle = MLEDisambiguator.pretrained()
tokenizer = MorphologicalTokenizer(mle, scheme='d3tok', split=True)

def preprocess_text(text):
   if pd.isna(text):  #NaN cases
        return  str(text)


   sent_norm = normalize_unicode(text)
   sent_dediac = dediac_ar(sent_norm)
   sent_split = sent_dediac.split()
   tokens = tokenizer.tokenize(sent_split)
   return tokens

In [ ]:
def count_sent(text):
   if pd.isna(text):  #NaN cases
        return  str(text)

   sent_norm = normalize_unicode(text)
   sent_split = sent_norm.split()
   tokens = tokenizer.tokenize(sent_split)
   return tokens

In [ ]:
def count_words(sentence):
    if isinstance(sentence, str):
        return len(sentence.split())
    else:
        return 0

concatenated_df['word_count'] = concatenated_df['text'].apply(count_words)

print(concatenated_df[['text', 'word_count']])

                                                    text  word_count
0      توجيه المجلس الأول بشأن وضع قواعد مشتركة معينة...          18
1                      مع مراعاة رأي البرلمان الأوروبي ؛           6
2                                 لقد اعتمد هذا التوجيه:           4
3      2. يجب إعفاء أنواع النقل المدرجة في الملحق الأ...          15
4                                                القسم 2           2
...                                                  ...         ...
25285                                    من 23 شباط 2004           4
25286  مع مراعاة الطلب الذي أدلت به النمسا يوم 30 حزي...          11
25287  (3) التدابير المنصوص عليها في هذا النظام هي وف...          22
25288  1. تم منح الاستثناءات التالية من أحكام نظام (C...          12
25289  ج) تمنح لوكسمبورغ استثناءات فيما يتعلق بتقديم ...          22

[25290 rows x 2 columns]


In [ ]:
total_count = 0

for count in concatenated_df['word_count']:
    total_count += count

avg = total_count / len(concatenated_df['word_count'])

print(f"Average word count: {avg}")


Average word count: 27.27105575326216


In [ ]:
fdist_dict = {}

for index, row in concatenated_df.iterrows():
    text = preprocess_text(row['text'])
    label = row['labels']

    if label not in fdist_dict:
        fdist_dict[label] = FreqDist()

    fdist_dict[label].update(ngrams(text, 3))

for label, fdist in fdist_dict.items():
    print(f"Top 10 Trigrams for {label}:")
    print(fdist.most_common(10))


Top 10 Trigrams for MT:
[(('ال+', 'دول', 'ال+'), 1846), (('+ها', 'في', 'ال+'), 1817), (('دول', 'ال+', 'أعضاء'), 1792), (('في', 'ال+', 'مادة'), 1673), (('ال+', 'منصوص', 'على'), 1436), (('وفقا', 'ل+', 'ال+'), 1287), (('ال+', 'مشار', 'إلى'), 1262), (('مشار', 'إلى', '+ها'), 996), (('منصوص', 'على', '+ها'), 990), (('على', '+ها', 'في'), 972)]
Top 10 Trigrams for HT:
[(('ال+', 'دول', 'ال+'), 1877), (('دول', 'ال+', 'أعضاء'), 1842), (('+ها', 'في', 'ال+'), 1380), (('في', 'ال+', 'مادة'), 1316), (('ال+', 'منصوص', 'على'), 1154), (('ال+', 'مشار', 'إلى'), 947), (('وفقا', 'ل+', 'ال+'), 906), (('منصوص', 'على', '+ها'), 869), (('على', '+ها', 'في'), 763), (('مشار', 'إلى', '+ها'), 724)]


In [ ]:
pMT = fdist_dict.get('MT', FreqDist())
pHT = fdist_dict.get('HT', FreqDist())

def classify_sentence(test_sentence, pMT, pHT):
    test_sentence = preprocess_text(test_sentence)

    likelihood_HT = sum([math.log(pHT.get(trigram, 1e-10)) for trigram in ngrams(test_sentence, 3, pad_left=True, pad_right=True)])
    likelihood_MT = sum([math.log(pMT.get(trigram, 1e-10)) for trigram in ngrams(test_sentence, 3, pad_left=True, pad_right=True)])
    if likelihood_HT > likelihood_MT:
        classification = "HT"
    else:
        classification = "MT"

    return classification

success_count = 0
classifications = []

for test_sentence, true_label in zip(test_df['text'], test_df['labels']):
    classification = classify_sentence(test_sentence, pMT, pHT)
    classifications.append(classification)  # Store classifications

    if classification == true_label:
        success_count += 1

accuracy = success_count / len(test_df['text']) if len(test_df['text']) > 0 else 0
print(f"Accuracy: {accuracy}")


Accuracy: 0.649561053471668


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

true_labels = test_df['labels'].tolist()
predicted_labels = classifications

accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy}")

precision = precision_score(true_labels, predicted_labels, pos_label='HT')
print(f"Precision (HT): {precision}")

recall = recall_score(true_labels, predicted_labels, pos_label='HT')
print(f"Recall (HT): {recall}")

f1 = f1_score(true_labels, predicted_labels, pos_label='HT')
print(f"F1 Score (HT): {f1}")


Accuracy: 0.649561053471668
Precision (HT): 0.7660420215786485
Recall (HT): 0.4306464485235435
F1 Score (HT): 0.5513436190865434


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

true_labels = test_df['labels'].tolist()
predicted_labels = classifications

accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy}")

report = classification_report(true_labels, predicted_labels, target_names=['HT', 'MT'])
print("Classification Report:\n", report)


Accuracy: 0.649561053471668
Classification Report:
               precision    recall  f1-score   support

          HT       0.77      0.43      0.55      6265
          MT       0.60      0.87      0.71      6265

    accuracy                           0.65     12530
   macro avg       0.69      0.65      0.63     12530
weighted avg       0.69      0.65      0.63     12530



In [ ]:
for prediction, true_label in zip(classifications, test_df['labels']):
    print(f"Prediction: {prediction}, True Label: {true_label}")


In [11]:
!git branch --set-upstream-to=origin/main
!git config --global user.email "humaid.alblooshi@mbzuai.ac.ae"
!git config --global user.name "Humaid Alblooshi"

error: the requested upstream branch 'origin/main' does not exist
hint: 
hint: If you are planning on basing your work on an upstream
hint: branch that already exists at the remote, you may need to
hint: run "git fetch" to retrieve it.
hint: 
hint: If you are planning to push out a new local branch that
hint: will track its remote counterpart, you may want to use
hint: "git push -u" to set the upstream config as you push.


In [15]:
!git clone https://github.com/humaidalblooshi/Automatic-detection-of-machine-translated-Arabic-Text-at-the-sentence-level.git

%cd Automatic-detection-of-machine-translated-Arabic-Text-at-the-sentence-level

!git config --global user.name "Humaid Alblooshi"
!git config --global user.email "humaid.alblooshi@mbzuai.ac.ae"

!git fetch

!git branch --set-upstream-to=origin/main main

!touch README.md
!git add .
!git commit -m "Initial commit"

!git push origin HEAD:main


Cloning into 'Automatic-detection-of-machine-translated-Arabic-Text-at-the-sentence-level'...
/content/Automatic-detection-of-machine-translated-Arabic-Text-at-the-sentence-level/Automatic-detection-of-machine-translated-Arabic-Text-at-the-sentence-level/Automatic-detection-of-machine-translated-Arabic-Text-at-the-sentence-level/Automatic-detection-of-machine-translated-Arabic-Text-at-the-sentence-level/Automatic-detection-of-machine-translated-Arabic-Text-at-the-sentence-level/Automatic-detection-of-machine-translated-Arabic-Text-at-the-sentence-level/Automatic-detection-of-machine-translated-Arabic-Text-at-the-sentence-level/Automatic-detection-of-machine-translated-Arabic-Text-at-the-sentence-level
fatal: branch 'main' does not exist
[main (root-commit) dfcb351] Initial commit
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 README.md
fatal: could not read Username for 'https://github.com': No such device or address


In [9]:
# # !git clone https://github.com/humaidalblooshi/Automatic-detection-of-machine-translated-Arabic-Text-at-the-sentence-level.git
# # %cd Automatic-detection-of-machine-translated-Arabic-Text-at-the-sentence-level
!git add .
!git commit -m "First commit"
!git push origin main

On branch main
Your branch is based on 'origin/main', but the upstream is gone.
  (use "git branch --unset-upstream" to fixup)

nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address
